**GROUP 1:** 
* OPC = 1 + HPV = 1

**GROUP 2:** 
* OPC = 1 + HPV = 0
* OPC = 0

In [1]:
import numpy as np
import pandas as pd

In [34]:
#path_to_targets = './../../data_source/to_analysis/original_images/dfs_original_images.csv'
path_to_features = './../../data_source/to_analysis/original_images/all_features_original_images.csv'
#path_to_results_a = './../../data_source/to_analysis/hpv_splitting/all_features_original_images_hpv_group_a.csv'
#path_to_results_b = './../../data_source/to_analysis/hpv_splitting/all_features_original_images_hpv_group_b.csv'
#path_to_targets_a = './../../data_source/to_analysis/hpv_splitting/dfs_original_images_hpv_group_a.csv'
#path_to_targets_b = './../../data_source/to_analysis/hpv_splitting/dfs_original_images_hpv_group_b.csv'

#path_to_targets = './../../data_source/to_analysis/removed_broken_slices/dfs_removed_broken_slices.csv'
#path_to_features = './../../data_source/to_analysis/removed_broken_slices/all_features_removed_broken_slices.csv'
#path_to_results_a = './../../data_source/to_analysis/hpv_splitting/all_features_removed_broken_slices_hpv_group_a.csv'
#path_to_results_b = './../../data_source/to_analysis/hpv_splitting/all_features_removed_broken_slices_hpv_group_b.csv'
#path_to_targets_a = './../../data_source/to_analysis/hpv_splitting/dfs_removed_broken_slices_hpv_group_a.csv'
#path_to_targets_b = './../../data_source/to_analysis/hpv_splitting/dfs_removed_broken_slices_hpv_group_b.csv'

#path_to_targets = './../../data_source/to_analysis/original_images/dfs_original_images.csv'
#path_to_features = './../../data_source/to_analysis/compressed_features/all_features_orig_images_icc.csv'
#path_to_results_a = './../../data_source/to_analysis/hpv_splitting/all_features_orig_images_icc_hpv_group_a.csv'
#path_to_results_b = './../../data_source/to_analysis/hpv_splitting/all_features_orig_images_icc_hpv_group_b.csv'
#path_to_targets_a = './../../data_source/to_analysis/hpv_splitting/dfs_original_images_hpv_group_a.csv'
#path_to_targets_b = './../../data_source/to_analysis/hpv_splitting/dfs_original_images_hpv_group_b.csv'

path_to_targets = './../../data_source/to_analysis/original_images/dfs_original_images.csv'
#path_to_features = './../../data_source/to_analysis/compressed_features/all_features_orig_images_icc_dropped.csv'
path_to_results_a = './../../data_source/to_analysis/hpv_splitting/all_features_orig_images_icc_dropped_hpv_group_a.csv'
path_to_results_b = './../../data_source/to_analysis/hpv_splitting/all_features_orig_images_icc_dropped_hpv_group_b.csv'
path_to_targets_a = './../../data_source/to_analysis/hpv_splitting/dfs_orig_images_hpv_group_a.csv'
path_to_targets_b = './../../data_source/to_analysis/hpv_splitting/dfs_orig_images_hpv_group_b.csv'

In [35]:
y = pd.read_csv(path_to_targets, index_col=0)

In [36]:
X = pd.read_csv(path_to_features, index_col=0)
patient_id = X.index

X.shape

(198, 513)

In [5]:
opc = pd.read_excel('./../../data_source/clinical/ops.xlsx', index_col=0)
opc = opc.loc[patient_id, :]
opc = np.squeeze(opc.values)
opc.shape

(198,)

In [6]:
clinical = pd.read_excel('../../data_source/clinical/clinical_params.xlsx', index_col=0)
clinical = clinical.loc[patient_id, :]
hpv = np.squeeze(clinical.loc[:, 'HPV-status'].values)
hpv.shape

(198,)

In [7]:
# * HPV realted: HPV+ and OPC (Einar: n=93).
# * HPV unrelated: HPV- and OPC, or non-OPC regardless of HPV status (Einar: n=73).
group_a_idx = []
group_b_idx = []
for num, (_opc, _hpv) in enumerate(zip(opc, hpv)):
    if _opc == 1 and _hpv == 1:
        group_a_idx.append(patient_id[num])
    elif (_opc == 1 and _hpv == 0) or (_opc == 0):
        group_b_idx.append(patient_id[num])
        
len(group_a_idx), len(group_b_idx), len(group_a_idx) + len(group_b_idx)

(82, 67, 149)

In [8]:
np.where(np.isin(group_a_idx, group_b_idx))

(array([], dtype=int64),)

In [44]:
X_grp_a = X.loc[group_a_idx, :]
X_grp_b = X.loc[group_b_idx, :]

In [10]:
X_grp_a.to_csv(path_to_results_a)
X_grp_b.to_csv(path_to_results_b)

In [11]:
y.loc[group_a_idx, :].to_csv(path_to_targets_a)
y.loc[group_b_idx, :].to_csv(path_to_targets_b)

In [12]:
y_grp_a = np.squeeze(y.loc[group_a_idx, :].values)
y_grp_b = np.squeeze(y.loc[group_b_idx, :].values)

In [13]:
print('No info rate group a: ', max(sum(y_grp_a == 0), sum(y_grp_a == 1)) / np.size(y_grp_a))
print('No info rate group b: ', max(sum(y_grp_b == 0), sum(y_grp_b == 1)) / np.size(y_grp_b))

No info rate group a:  0.7317073170731707
No info rate group b:  0.5373134328358209


In [55]:
# HPV Related
X_grp_a.filter(regex='original_shape_MeshVolume').max() / 1000

original_shape_MeshVolume    145.807833
dtype: float64

In [57]:
# HPV Unrelated
X_grp_b.filter(regex='original_shape_MeshVolume').max() / 1000

original_shape_MeshVolume    284.854875
dtype: float64

In [29]:
X_grp_a.shape, y_grp_a.shape, X_grp_b.shape, y_grp_b.shape

((82, 152), (82,), (67, 152), (67,))

In [30]:
82 / 152, 67 / 152, 198 / 351

(0.5394736842105263, 0.4407894736842105, 0.5641025641025641)

In [33]:
X_grp_b.filter(regex='Charlson')

,Charlson_1,Charlson_2,Charlson_3,Charlson_4,Charlson_5
5,0,0,0,0,0
11,1,0,0,0,0
12,0,1,0,0,0
15,0,0,0,1,0
16,0,1,0,0,0
23,0,0,0,0,0
24,0,0,0,0,0
31,1,0,0,0,0
35,0,0,0,0,0
36,0,0,0,0,0
